In [1]:
# import libraries necessary for data preprocessing 
import gym
from cv2 import cv2
import time
import numpy as np
from tqdm import tqdm
import os
import random

In [2]:
# create a random weights array once to be used to bring the 'observation' to 1000/256/any number (has to be consistent) units.
sizeOfArray = 256

if os.path.exists('weights_'+str(sizeOfArray)+'.npy'):
    array_weights_for_observation = np.load('weights_'+str(sizeOfArray)+'.npy')
else:
    array_weights_for_observation = np.random.rand(1,sizeOfArray)
    np.save('weights_'+str(sizeOfArray)+'.npy',array_weights_for_observation)

array_weights_for_observation = array_weights_for_observation.reshape(sizeOfArray)

In [12]:
X = []
X_obs = []
X_img = []
Y = []
initial_games = 1000  # Number of games to collect the data set. 
target_score = 1200  # minimum treshold score to save the game to dataset

In [13]:
env_name = "Carnival-ram-v0"   #one example name of the environment.

In [ ]:
#env_name = "Boxing-ram-v0"
env = gym.make(env_name)
for _ in tqdm(range(initial_games)):
    env.reset()
    score = 0
    prev_obs = []
    X_obs = []
    Y_temp = []
    while True:
        env.render()
        #time.sleep(0.05)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        action = env.action_space.sample() # your agent here (this takes random actions)
        observation, reward, done, info = env.step(action)
        
        
        if len(observation) > sizeOfArray: # get all observation array to size of 1000/256/any unit before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation.append([0]*(sizeOfArray-(len(observation)%sizeOfArray)))
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
            
        #print(action)
        if prev_obs != []:
            temp_Y = [0] * 10
            temp_Y[action] = 1
            Y_temp.append(temp_Y)
            score += reward
            X_obs.append(img)
            X_obs.append(prev_obs)
            #X.append(score)
        prev_obs = list(observation)
        
        if done:
            #print(score)
            break
    if score >= target_score:
        X.append(X_obs)
        Y.append(Y_temp)




In [ ]:
X[0][1].shape

In [17]:
# Converting the data in a convenient format for Neural Network to process later 
len(X)  
X = np.array(X)
Y = np.array(Y)
X_final = []
Y_final = []
for i in tqdm(range(int(X.size))):
    for j in range(len(X[i])):
        X_final.append(X[i][j])
        
    for j in range(len(Y[i])):
        Y_final.append(Y[i][j])

100%|██████████| 10/10 [00:00<00:00, 1684.66it/s]


In [18]:
X = np.array(X_final)
Y = np.array(Y_final)
X_final = []
Y_final = []

In [19]:
X = X.reshape(int(X.size/2),2)

In [20]:
Y.shape

(4733, 10)

In [10]:
np.save(env_name+'_X_10.npy',X)   # Save the dataset for the current game for reuse
np.save(env_name+'_Y_10.npy',Y)

In [21]:
# Testing data
np.save(env_name+'_X_10_Test.npy',X)
np.save(env_name+'_Y_10_Test.npy',Y)

In [14]:
X = np.load(env_name+'_X_10.npy')  # Load the saved dataset
Y = np.load(env_name+'_Y_10.npy')

In [10]:
np.save(env_name+'_X.npy',X)
np.save(env_name+'_Y.npy',Y)

In [4]:
X = np.load(env_name+'_X.npy')
Y = np.load(env_name+'_Y.npy')

# Training the Neural Network Models

In [6]:
# Tensorflow and TFLearn libraries required to build, train and test the neural network.

import tensorflow as tf
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# Neural Network Model 1

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-1'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing

convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')


convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2) # Top K metric can be set here.
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
train_X = X[:-2000]
test_X = X[-2000:]
train_Y = Y[:-2000]
test_Y = Y[-2000:]

X_1 = np.array([i[0] for i in train_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X_2 = np.array([i[1] for i in train_X]).reshape(-1,len(X[0][1]))
#X_3 = np.array([i[2] for i in train_X]).reshape(-1,1)

test_X_1 = np.array([i[0] for i in test_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_X_2 = np.array([i[1] for i in test_X]).reshape(-1,len(X[0][1]))
#test_X_3 = np.array([i[2] for i in test_X]).reshape(-1,1)

model.fit({'input-1': X_1, 'input-2': X_2}, {'targets': train_Y}, n_epoch=8, validation_set=({'input-1': test_X_1, 'input-2': test_X_2}, {'targets': test_Y}), 
     show_metric=True, run_id=MODEL_NAME, batch_size = 16)

model.save(MODEL_NAME)

In [ ]:
# Neural Network Model 2

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-2'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = tf.reshape(convnet, shape=[-1, 15, 2000])
convnet =  tflearn.lstm(convnet, 1024, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 512, activation='relu', return_seq=True)


convnet =  tflearn.lstm(convnet, 256, activation='relu')

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 256, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')


convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2) # Top K metric can be set here
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
train_X = X[:-2000]
test_X = X[-2000:]
train_Y = Y[:-2000]
test_Y = Y[-2000:]

X_1 = np.array([i[0] for i in train_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X_2 = np.array([i[1] for i in train_X]).reshape(-1,len(X[0][1]))
#X_3 = np.array([i[2] for i in train_X]).reshape(-1,1)

test_X_1 = np.array([i[0] for i in test_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_X_2 = np.array([i[1] for i in test_X]).reshape(-1,len(X[0][1]))
#test_X_3 = np.array([i[2] for i in test_X]).reshape(-1,1)

model.fit({'input-1': X_1, 'input-2': X_2}, {'targets': train_Y}, n_epoch=8, validation_set=({'input-1': test_X_1, 'input-2': test_X_2}, {'targets': test_Y}), 
     show_metric=True, run_id=MODEL_NAME, batch_size = 16)

model.save(MODEL_NAME)

In [ ]:
# Neural Network Model 3

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-3'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing

convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')

convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2)
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
train_X = X[:-2000]
test_X = X[-2000:]
train_Y = Y[:-2000]
test_Y = Y[-2000:]

X_1 = np.array([i[0] for i in train_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X_2 = np.array([i[1] for i in train_X]).reshape(-1,len(X[0][1]))
#X_3 = np.array([i[2] for i in train_X]).reshape(-1,1)

test_X_1 = np.array([i[0] for i in test_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_X_2 = np.array([i[1] for i in test_X]).reshape(-1,len(X[0][1]))
#test_X_3 = np.array([i[2] for i in test_X]).reshape(-1,1)

model.fit({'input-1': X_1, 'input-2': X_2}, {'targets': train_Y}, n_epoch=8, validation_set=({'input-1': test_X_1, 'input-2': test_X_2}, {'targets': test_Y}), 
     show_metric=True, run_id=MODEL_NAME, batch_size = 16)

model.save(MODEL_NAME)

In [ ]:
# Neural Network Model 4

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-4'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)


convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing

convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 256, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')


convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')
#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2) # Top K metric is set here
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
train_X = X[:-2000]
test_X = X[-2000:]
train_Y = Y[:-2000]
test_Y = Y[-2000:]

X_1 = np.array([i[0] for i in train_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X_2 = np.array([i[1] for i in train_X]).reshape(-1,len(X[0][1]))
#X_3 = np.array([i[2] for i in train_X]).reshape(-1,1)

test_X_1 = np.array([i[0] for i in test_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_X_2 = np.array([i[1] for i in test_X]).reshape(-1,len(X[0][1]))
#test_X_3 = np.array([i[2] for i in test_X]).reshape(-1,1)

model.fit({'input-1': X_1, 'input-2': X_2}, {'targets': train_Y}, n_epoch=8, validation_set=({'input-1': test_X_1, 'input-2': test_X_2}, {'targets': test_Y}), 
     show_metric=True, run_id=MODEL_NAME, batch_size = 16)

model.save(MODEL_NAME)

In [ ]:
# Neural Network Model 5

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-5'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = tf.reshape(convnet, shape=[-1, 16, 2])
convnet =  tflearn.lstm(convnet, 1024, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 512, activation='relu', return_seq=True)


convnet =  tflearn.lstm(convnet, 256, activation='relu')

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 256, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')
convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2)
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
train_X = X[:-2000]
test_X = X[-2000:]
train_Y = Y[:-2000]
test_Y = Y[-2000:]

X_1 = np.array([i[0] for i in train_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X_2 = np.array([i[1] for i in train_X]).reshape(-1,len(X[0][1]))
#X_3 = np.array([i[2] for i in train_X]).reshape(-1,1)

test_X_1 = np.array([i[0] for i in test_X]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_X_2 = np.array([i[1] for i in test_X]).reshape(-1,len(X[0][1]))
#test_X_3 = np.array([i[2] for i in test_X]).reshape(-1,1)

model.fit({'input-1': X_1, 'input-2': X_2}, {'targets': train_Y}, n_epoch=8, validation_set=({'input-1': test_X_1, 'input-2': test_X_2}, {'targets': test_Y}), 
     show_metric=True, run_id=MODEL_NAME, batch_size = 16)

model.save(MODEL_NAME)

# TESTING

In [ ]:
# Test the Neural Network model on the gym env game in real time.

In [ ]:
# Neural Network Model 1

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-1'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing

convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')


convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2) # Top K metric can be set here.
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')



env = gym.make(env_name)
action = 0
print(action)
j=0
for _ in range(10):  # Play for 10 turns for the respective game
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        if action > 1:
            action = 1
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000/256/required unit before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            #action = int(action - random.randint(0,10))
            #print(action)
            #print(model.predict([img,observation_1]))
        prev_obs = list(observation)
        j+=1
        if done:
            print(score)
            break
            
'''
# Alternate Way to test the Model. Test the Neural Network on the saved testing dataset frames 

count=0
for i in tqdm(range(X.shape[0])):
    temp = np.argmax(model.predict([X[i][0].reshape(-1,IMG_SIZE,IMG_SIZE,3), np.array(X[i][1]).reshape(-1,len(X[i][1]))]))
    if(temp == np.argmax(Y[i])):
        count+=1

print("Accuracy is "+ str(float((float(count)/float(X.shape[0]))*100))+ "%")
print(count)
'''

In [ ]:
# Neural Network Model 2

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-2'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = tf.reshape(convnet, shape=[-1, 15, 2000])
convnet =  tflearn.lstm(convnet, 1024, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 512, activation='relu', return_seq=True)


convnet =  tflearn.lstm(convnet, 256, activation='relu')

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 256, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')


convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2) # Top K metric can be set here
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
env = gym.make(env_name)
action = 0
print(action)
j=0
for _ in range(10):  # Play for 10 turns for the respective game
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        if action > 1:
            action = 1
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000/256/required unit before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            #action = int(action - random.randint(0,10))
            #print(action)
            #print(model.predict([img,observation_1]))
        prev_obs = list(observation)
        j+=1
        if done:
            print(score)
            break
            
'''
# Alternate Way to test the Model. Test the Neural Network on the saved testing dataset frames 

count=0
for i in tqdm(range(X.shape[0])):
    temp = np.argmax(model.predict([X[i][0].reshape(-1,IMG_SIZE,IMG_SIZE,3), np.array(X[i][1]).reshape(-1,len(X[i][1]))]))
    if(temp == np.argmax(Y[i])):
        count+=1

print("Accuracy is "+ str(float((float(count)/float(X.shape[0]))*100))+ "%")
print(count)
'''

In [ ]:
# Neural Network Model 3

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-3'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing

convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')

convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2)
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
env = gym.make(env_name)
action = 0
print(action)
j=0
for _ in range(10):  # Play for 10 turns for the respective game
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        if action > 1:
            action = 1
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000/256/required unit before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            #action = int(action - random.randint(0,10))
            #print(action)
            #print(model.predict([img,observation_1]))
        prev_obs = list(observation)
        j+=1
        if done:
            print(score)
            break
            
'''
# Alternate Way to test the Model. Test the Neural Network on the saved testing dataset frames 

count=0
for i in tqdm(range(X.shape[0])):
    temp = np.argmax(model.predict([X[i][0].reshape(-1,IMG_SIZE,IMG_SIZE,3), np.array(X[i][1]).reshape(-1,len(X[i][1]))]))
    if(temp == np.argmax(Y[i])):
        count+=1

print("Accuracy is "+ str(float((float(count)/float(X.shape[0]))*100))+ "%")
print(count)
'''

In [ ]:
# Neural Network Model 4

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-4'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)


convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing

convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 256, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')


convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')
#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2) # Top K metric is set here
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
env = gym.make(env_name)
action = 0
print(action)
j=0
for _ in range(10):  # Play for 10 turns for the respective game
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        if action > 1:
            action = 1
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000/256/required unit before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            #action = int(action - random.randint(0,10))
            #print(action)
            #print(model.predict([img,observation_1]))
        prev_obs = list(observation)
        j+=1
        if done:
            print(score)
            break

'''
# Alternate Way to test the Model. Test the Neural Network on the saved testing dataset frames 

count=0
for i in tqdm(range(X.shape[0])):
    temp = np.argmax(model.predict([X[i][0].reshape(-1,IMG_SIZE,IMG_SIZE,3), np.array(X[i][1]).reshape(-1,len(X[i][1]))]))
    if(temp == np.argmax(Y[i])):
        count+=1

print("Accuracy is "+ str(float((float(count)/float(X.shape[0]))*100))+ "%")
print(count)
'''

In [ ]:
# Neural Network Model 5

tf.reset_default_graph()
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'Neural_Network_Model-5'


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input-1')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = tf.reshape(convnet, shape=[-1, 16, 2])
convnet =  tflearn.lstm(convnet, 1024, activation='relu', return_seq=True)

convnet =  tflearn.lstm(convnet, 512, activation='relu', return_seq=True)


convnet =  tflearn.lstm(convnet, 256, activation='relu')

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

#Input 2 processing
convnet1 = input_data(shape=[None, len(array_weights_for_observation)], name='input-2')

convnet1 = tf.reshape(convnet1, shape=[-1, 64, 4])

convnet1 =  tflearn.lstm(convnet1, 256, activation='relu', return_seq=True)

convnet1 =  tflearn.lstm(convnet1, 128, activation='relu')
convnet_partial_merge = fully_connected(convnet1, 64, activation='relu')

#Merge layer
convnet_partial_merge = tflearn.layers.merge_ops.merge ([convnet,convnet_partial_merge], 'elemwise_mul', axis=1, name='Merge')
convnet_partial_merge = dropout(convnet_partial_merge, 0.8)

print(convnet_partial_merge)

convnet = fully_connected(convnet_partial_merge, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
print(convnet.shape)
convnet = fully_connected(convnet, 10, activation='softmax')
top_k = tflearn.metrics.Top_k(2)
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets',metric=top_k)

model = tflearn.DNN(convnet, tensorboard_dir='log_new')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


    
env = gym.make(env_name)
action = 0
print(action)
j=0
for _ in range(10):  # Play for 10 turns for the respective game
    env.reset()
    score = 0
    prev_obs = []
    while True:
        env.render()
        time.sleep(0.03)
        img = env.render(mode = 'rgb_array')
        img = cv2.resize(img, (100, 100))
        if action > 1:
            action = 1
        observation, reward, done, info = env.step(action)
        observation = observation.flatten()
        
        if len(observation) > sizeOfArray:     # get all observation array to size of 1000/256/required unit before saving
            if len(observation) % sizeOfArray == 0:
                observation = np.array(observation)
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
            else:
                observation = np.array(observation)
                observation = np.append(observation,([0]*(sizeOfArray-(len(observation)%sizeOfArray))))
                observation = observation.reshape(-1,int(observation.size/sizeOfArray)).mean(axis=1)
                observation = np.convolve(np.array(observation),array_weights_for_observation,'same')
                
        else:
            observation = np.array(observation)
            #print(observation.shape)
            #print(array_weights_for_observation.shape)
            observation = np.convolve(observation,array_weights_for_observation,'same')
        
        if prev_obs != []:
            img = img.reshape(-1,IMG_SIZE,IMG_SIZE,3)
            observation_1 = np.array(prev_obs).reshape(-1,len(prev_obs))
            score += reward
            #score_1 = np.array(score).reshape(-1,1)
            action = np.argmax(model.predict([img,observation_1]))
            #action = int(action - random.randint(0,10))
            #print(action)
            #print(model.predict([img,observation_1]))
        prev_obs = list(observation)
        j+=1
        if done:
            print(score)
            break
            
'''
# Alternate Way to test the Model. Test the Neural Network on the saved testing dataset frames 

count=0
for i in tqdm(range(X.shape[0])):
    temp = np.argmax(model.predict([X[i][0].reshape(-1,IMG_SIZE,IMG_SIZE,3), np.array(X[i][1]).reshape(-1,len(X[i][1]))]))
    if(temp == np.argmax(Y[i])):
        count+=1

print("Accuracy is "+ str(float((float(count)/float(X.shape[0]))*100))+ "%")
print(count)
'''